In [1]:
import os
import glob
from PIL import Image, ImageOps

input_dirname = "/content/dataset/predicted_auto/*"
output_dirname = "/content/dataset/gen/auto"
files = glob.glob(input_dirname)
reshaped_size = (128, 128)

for i, file in enumerate(files):
    index = i + 1
    try:
        image = Image.open(file)
    except IOError:
        pass
    reshaped = ImageOps.fit(image, reshaped_size, Image.NEAREST)
    converted = reshaped.convert('RGB')
    converted.save(os.path.join(output_dirname, f'{index}.jpg'))
    print(f'{index}: {file} was saved.')

1: /content/dataset/predicted_auto/305.jpg was saved.
2: /content/dataset/predicted_auto/977.jpg was saved.
3: /content/dataset/predicted_auto/113.jpg was saved.
4: /content/dataset/predicted_auto/1258.jpg was saved.
5: /content/dataset/predicted_auto/865.jpg was saved.
6: /content/dataset/predicted_auto/1400.jpg was saved.
7: /content/dataset/predicted_auto/1495.jpg was saved.
8: /content/dataset/predicted_auto/333.jpg was saved.
9: /content/dataset/predicted_auto/512.jpg was saved.
10: /content/dataset/predicted_auto/710.jpg was saved.
11: /content/dataset/predicted_auto/410.jpg was saved.
12: /content/dataset/predicted_auto/1374.jpg was saved.
13: /content/dataset/predicted_auto/407.jpg was saved.
14: /content/dataset/predicted_auto/1122.jpg was saved.
15: /content/dataset/predicted_auto/1446.jpg was saved.
16: /content/dataset/predicted_auto/1217.jpg was saved.
17: /content/dataset/predicted_auto/1282.jpg was saved.
18: /content/dataset/predicted_auto/460.jpg was saved.
19: /conten

In [5]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session

import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from pathlib import Path

from IPython.display import Image
from keras.utils.vis_utils import model_to_dot

np.random.seed(0)
np.random.RandomState(0)
tf.random.set_seed(0)

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
session = tf.compat.v1.Session(config=config)
set_session(session)

root_dir = "/content/dataset/gen"

class DCGAN():
    def __init__(self):

        self.class_names = os.listdir(root_dir)

        self.shape = (128, 128, 3)
        self.z_dim = 100

        optimizer = Adam(lr=0.0002, beta_1=0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        self.generator = self.build_generator()
        # self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        z = Input(shape=(self.z_dim,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        noise_shape = (self.z_dim,)

        model = Sequential()

        model.add(Dense(128 * 32 * 32, activation="relu", input_shape=noise_shape))
        model.add(Reshape((32, 32, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        Image(model_to_dot(model).create(prog='dot', format='png'))

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        img_shape = self.shape

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def build_combined(self):
        self.discriminator.trainable = False
        model = Sequential([self.generator, self.discriminator])

        return model

    def train(self, iterations, batch_size=128, save_interval=50, model_interval=1000, check_noise=None, r=5, c=5):

        X_train, labels = self.load_imgs()

        half_batch = int(batch_size / 2)

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        for iteration in range(iterations):

            # ------------------
            # Training Discriminator
            # -----------------
            idx = np.random.randint(0, X_train.shape[0], half_batch)

            imgs = X_train[idx]

            noise = np.random.uniform(-1, 1, (half_batch, self.z_dim))

            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # -----------------
            # Training Generator
            # -----------------

            noise = np.random.uniform(-1, 1, (batch_size, self.z_dim))

            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100 * d_loss[1], g_loss))

            model_dir = Path('ganmodels')
            model_dir.mkdir(exist_ok=True)
            if iteration % save_interval == 0:
                self.save_imgs(iteration, check_noise, r, c)
                start = np.expand_dims(check_noise[0], axis=0)
                end = np.expand_dims(check_noise[1], axis=0)
                resultImage = self.visualizeInterpolation(start=start, end=end)
                cv2.imwrite("images/latent/" + "latent_{}.png".format(iteration), resultImage)
                if iteration % model_interval == 0:
                    self.generator.save(str(model_dir)+"/dcgan-{}-iter.h5".format(iteration))

    def save_imgs(self, iteration, check_noise, r, c):
        noise = check_noise
        gen_imgs = self.generator.predict(noise)

        # 0-1 rescale
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig('images/gen_imgs/doraemon_%d.png' % iteration)

        plt.close()

    def load_imgs(self):

        img_paths = []
        labels = []
        images = []
        for cl_name in self.class_names:
            img_names = os.listdir(os.path.join(root_dir, cl_name))
            for img_name in img_names:
                img_paths.append(os.path.abspath(os.path.join(root_dir, cl_name, img_name)))
                hot_cl_name = self.get_class_one_hot(cl_name)
                labels.append(hot_cl_name)

        for img_path in img_paths:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)

        images = np.array(images)

        return (np.array(images), np.array(labels))

    def get_class_one_hot(self, class_str):
        label_encoded = self.class_names.index(class_str)

        label_hot = np_utils.to_categorical(label_encoded, len(self.class_names))
        label_hot = label_hot

        return label_hot

    def visualizeInterpolation(self, start, end, save=True, nbSteps=10):
        print("Generating interpolations...")

        steps = nbSteps
        latentStart = start
        latentEnd = end

        startImg = self.generator.predict(latentStart)
        endImg = self.generator.predict(latentEnd)

        vectors = []

        alphaValues = np.linspace(0, 1, steps)
        for alpha in alphaValues:
            vector = latentStart * (1 - alpha) + latentEnd * alpha
            vectors.append(vector)

        vectors = np.array(vectors)

        resultLatent = None
        resultImage = None

        for i, vec in enumerate(vectors):
            gen_img = np.squeeze(self.generator.predict(vec), axis=0)
            gen_img = (0.5 * gen_img + 0.5) * 255
            interpolatedImage = cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR)
            interpolatedImage = interpolatedImage.astype(np.uint8)
            resultImage = interpolatedImage if resultImage is None else np.hstack([resultImage, interpolatedImage])

        return resultImage


if __name__ == '__main__':

    dcgan = DCGAN()
    r, c = 5, 5
    check_noise = np.random.uniform(-1, 1, (r * c, 100))
    dcgan.train(iterations=5000, batch_size=32, save_interval=1000,
                model_interval=5000, check_noise=check_noise, r=r,c=c)

ストリーミング出力は最後の 5000 行に切り捨てられました。
4 [D loss: 0.242334, acc.: 100.00%] [G loss: 0.086633]
5 [D loss: 0.079307, acc.: 100.00%] [G loss: 0.058073]
6 [D loss: 0.023379, acc.: 100.00%] [G loss: 0.025329]
7 [D loss: 0.079611, acc.: 96.88%] [G loss: 0.003347]
8 [D loss: 0.005392, acc.: 100.00%] [G loss: 0.001119]
9 [D loss: 0.005185, acc.: 100.00%] [G loss: 0.000649]
10 [D loss: 0.001992, acc.: 100.00%] [G loss: 0.000633]
11 [D loss: 0.001548, acc.: 100.00%] [G loss: 0.000378]
12 [D loss: 0.001430, acc.: 100.00%] [G loss: 0.000474]
13 [D loss: 0.001071, acc.: 100.00%] [G loss: 0.000283]
14 [D loss: 0.002141, acc.: 100.00%] [G loss: 0.000221]
15 [D loss: 0.001633, acc.: 100.00%] [G loss: 0.000116]
16 [D loss: 0.001078, acc.: 100.00%] [G loss: 0.000120]
17 [D loss: 0.002305, acc.: 100.00%] [G loss: 0.000102]
18 [D loss: 0.118482, acc.: 93.75%] [G loss: 0.000004]
19 [D loss: 0.034456, acc.: 100.00%] [G loss: 0.000001]
20 [D loss: 0.008639, acc.: 100.00%] [G loss: 0.000001]
21 [D loss: 0.004509, ac